In [1]:
import feedparser
import csv
import pandas as pd
import re

In [17]:
'Kommersant': 'https://www.kommersant.ru/RSS/corp.xml',
https://www.kommersant.ru/RSS/section-sport.xml
    https://www.kommersant.ru/RSS/news.xml
        https://www.kommersant.ru/RSS/corp.xml #материалы сайта коммерсант
            https://www.kommersant.ru/RSS/section-economics.xml
                https://www.kommersant.ru/RSS/section-culture.xml
                    https://www.kommersant.ru/RSS/section-style.xml
                        https://www.kommersant.ru/RSS/Autopilot_on.xml
                            https://www.kommersant.ru/RSS/section-hitech.xml
                                https://www.kommersant.ru/RSS/section-business.xml
                                    https://www.kommersant.ru/RSS/section-society.xml
                                        <link href="https://lenta.ru/rss/google-newsstand/main/" rel="alternate" type="application/rss+xml">
                                        <href="https://ria.ru/export/rss2/archive/index.xml">

SyntaxError: invalid syntax (588489674.py, line 1)

In [40]:
our_feeds = {'ria': 'https://ria.ru/export/rss2/archive/index.xml'} #пример словаря RSS-лент 
                                                                  #русскоязычных источников

f_all_news = '../datasets/show.csv' 
f_certain_news = '../datasets/show1.csv' #пример пути файла

vector1 = 'ДолЛАР|РубЛ|ЕвРО' #пример таргетов
vector2 = 'ЦБ|СбЕРбАНК|курс'

In [41]:
def check_url(url_feed): #функция получает линк на рсс ленту, возвращает        
# распаршенную ленту с помощью feedpaeser
    return feedparser.parse(url_feed)  
    
def getHeadlines(url_feed): #функция для получения заголовков новости
    headlines = []
    lenta = check_url (url_feed)
    for item_of_news in lenta['items']:
        headlines.append(item_of_news ['title'])
    return headlines

#def getDescriptions(url_feed): #функция для получения описания новости
#    descriptions = []
 #   lenta = check_url(url_feed)
  #  for item_of_news in lenta['items']:
   #     descriptions.append(item_of_news ['description'])
    #return descriptions

def getLinks(url_feed): #функция для получения ссылки на источник новости
    links = []
    lenta = check_url(url_feed)
    for item_of_news in lenta['items']:
        links.append(item_of_news ['link'])
    return links

def getDates(url_feed): #функция для получения даты публикации новости
    dates = []
    lenta = check_url(url_feed)
    for item_of_news in lenta['items']:
        dates.append(item_of_news ['published'])
    return dates

In [43]:
allheadlines = []
alldescriptions = []
alllinks = []
alldates = []
# Прогоняем наши URL и добавляем их в наши пустые списки
for key,url in our_feeds.items():
    allheadlines.extend( getHeadlines(url) )
    
#for key,url in our_feeds.items():
#    alldescriptions.extend( getDescriptions(url) )
    
for key,url in our_feeds.items():
    alllinks.extend( getLinks(url) )
    
for key,url in our_feeds.items():
    alldates.extend( getDates(url) )

In [44]:
def write_all_news(all_news_filepath): #функция для записи всех новостей в .csv, 
# возвращает нам этот датасет
    header = ['Title','Description','Links','Publication Date'] 

    with open(all_news_filepath, 'w', encoding='utf-8-sig') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')

        writer.writerow(i for i in header) 

        for a,b,c,d  in zip(allheadlines,alldescriptions,
                            alllinks, alldates):
            writer.writerow((a,b,c,d))


        df = pd.read_csv(all_news_filepath)
            
    return df

In [45]:
def looking_for_certain_news(all_news_filepath, certain_news_filepath, target1, target2): #функция для поиска, а затем записи
                #определенных новостей по таргета,
                #затем возвращает этот датасет
    df = pd.read_csv(all_news_filepath)
    
    result = df.apply(lambda x: x.str.contains(target1, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    result2 = df.apply(lambda x: x.str.contains(target2, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    new_df = df[result&result2]
        
    new_df.to_csv(certain_news_filepath
                     ,sep = '\t', encoding='utf-8-sig')
        
    return new_df

In [46]:
pd.set_option('display.max_colwidth', None)
write_all_news(f_all_news) 
#все новости

EmptyDataError: No columns to parse from file

In [124]:
looking_for_certain_news(f_all_news, f_certain_news, vector1, vector2) 

,Title,Description,Links,Publication Date
24,Работники держат курс на повышение // Готов ли российский бизнес платить сотрудникам за границей в валюте,"Сотрудники российских компаний, живущие за рубежом, просят привязать зарплаты к курсу валюты. Больше половины работодателей в августе получили просьбы и об индексации вознаграждения со стороны таких специалистов, показал опрос HeadHunter. Поводом стало резкое ослабление рубля. Как отмечается, 23% предприятий планируют пойти навстречу и повысить оклад удаленщикам, проживающим за границей. Столько же привяжут выплаты в рублях к курсу доллара или евро. Еще 23% не примут никаких мер.",https://www.kommersant.ru/doc/6212688,"Thu, 14 Sep 2023 19:16:03 +0300"


In [ ]:
good_news = f_all_news.conncat()